<a href="https://colab.research.google.com/github/kgautam2103/zoom_email_chat_extract_public/blob/main/zoom_email_chat_extract_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# when you run this code, you will get a link below
#Follow the link, copy the code, paste it in the box and then press Enter on the keyboard.
#This will authenticate you and will allow you to interact with Google Sheets and other Google apps in your notebook.
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
import sys
import re
import pandas as pd
import numpy as np
import traceback
from io import StringIO

wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1WhACIU6F7i9VwDxXoqzx6GEwutQI5HpIYl4v6H49-7Q/edit#gid=0')

input_sheet = wb.worksheet('sahajyogis')
inputdata = input_sheet.get_all_values()
inputdata_df = pd.DataFrame(inputdata, columns=['id','name','email'])
inputdata_df = inputdata_df.drop(inputdata_df.index[0])
ref_name_list = list(filter(None, inputdata_df['name'].tolist()))
ref_name_list = [i.lower() for i in ref_name_list]

from google.colab import files
chat_upload = files.upload()

value_iterator = iter(chat_upload.values()) 
chat_details = next(value_iterator)
chat_details = str(chat_details.decode('utf-8'))
chat_str = StringIO(chat_details)
chat_data = pd.read_csv(chat_str, sep=" : ",names=["details","chat"])
chat_data = chat_data.replace(to_replace='None', value=np.nan).dropna()
detail_list = chat_data["details"].values.tolist()
name_list=[]
for name_detail in detail_list:
  from_index = name_detail.find("From ")
  to_index = name_detail.find(" to")
  if to_index == -1:
      name = name_detail[from_index + 6:]
  else:
      name = name_detail[from_index + 6:to_index - 1]
  name_list.append(name)
  #print(from_index,to_index,name)
chat_data["name"] = name_list

chat_list = chat_data["chat"].values.tolist()
email_list = []
for chat in chat_list:
  atrate_index = chat.find("@")
  #print(atrate_index)
  if atrate_index == -1:
      email = ""
  else:
      space_index_before = chat[:atrate_index].rfind(" ")
      space_index_after = chat[atrate_index:].find(" ")
      if space_index_after == -1 and space_index_before == -1:
          email = chat
      else:
          #email = chat[space_index_before+1:space_index_after+1]
          if space_index_before == -1:
              email = chat[:atrate_index+space_index_after]
          elif space_index_after == -1:
              email = chat[space_index_before+1:]
          else:
              email = chat[space_index_before+1:atrate_index+space_index_after]
  email_list.append(email)

chat_data["email"] = email_list
extract_df = chat_data[["name","email","chat"]]
#final_df_email = extract_df.groupby("name").apply(lambda x: x["email"].tolist())
final_df_email = extract_df.groupby("name").apply(lambda x: [x["email"].tolist(), x["chat"].tolist()]).apply(pd.Series)
#final_df_email.columns=["name","email"]

#concat_df = pd.concat([final_df_email[0].reset_index(),final_df_email[1].reset_index()],axis=1)

merge_df = pd.merge(final_df_email[0].reset_index(),final_df_email[1].reset_index(),on="name")

final_df = merge_df.rename(columns={0:'email',1:'chat'}, inplace=False)

final_df['email'] = [(','.join(list(set(rx)))).strip(',')for rx in final_df['email']]

final_df['chat'] = [(', '.join(rx)).strip(',')for rx in final_df['chat']]

final_df['name'] = final_df['name'].apply(lambda x: re.sub('^sy[\.\s_-]*','',x,flags=re.IGNORECASE))\
  .apply(lambda x: re.sub('[\.\s_-]*sy$','',x,flags=re.IGNORECASE))

final_df = final_df[~(final_df.name).str.lower().isin(ref_name_list)]

final_df.to_csv('ZoomChat_Extract.csv') 
files.download("ZoomChat_Extract.csv")

#final_df

In [ ]:
import pandas as pd

wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1YHIfMt0OFUvrYPyXZHTKKGTvDjBp9fKiFgfoGL_XLxs/edit?ts=5ee5b2b7#gid=227936794')

poll1_sheet = wb.worksheet('MainPoll1')
poll1_data = poll1_sheet.get_all_values()
poll1_df = pd.DataFrame(poll1_data, columns=['event','name','poll1_answer'])
poll1_df = poll1_df.drop(poll1_df.index[0])

poll2_sheet = wb.worksheet('MainPoll2')
poll2_data = poll2_sheet.get_all_values()
poll2_df = pd.DataFrame(poll2_data, columns=['event','name','poll2_answer1','poll2_answer2','poll2_answer3'])
poll2_df = poll2_df.drop(poll2_df.index[0])
filter_event = input("enter event name like 20210626 APAC : ")

filter = poll1_df['event'] == filter_event
poll1_df_filtered = poll1_df.where(filter).dropna()
poll2_df_filtered = poll2_df.where(filter).dropna()

poll1_df_chat = pd.merge(final_df, poll1_df_filtered, on='name', how='outer')

Combined_poll_df_chat = pd.merge(poll1_df_chat, poll2_df_filtered, on='name', how='outer')

Combined_poll_df_chat_select = Combined_poll_df_chat[['name','email','chat','poll1_answer','poll2_answer1','poll2_answer2','poll2_answer3']]
#Combined_poll_df_chat_select.dropna(how='all')

Combined_poll_df_chat_select = Combined_poll_df_chat_select.replace(to_replace='None', value=np.nan).dropna(how='all')

Combined_poll_df_chat_select.to_csv('ZoomChat_Poll_Extract.csv') 
files.download("ZoomChat_Poll_Extract.csv")




enter event name like 20210626 APAC : 20210725 APAC


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>